In [ ]:
!pip install gfatpy==0.2.0

# Práctica de lidar Raman

Éste es el código que se utilizará para el desarrollo de la práctica de lidar Raman. Para ello el discente solo tendrá que modificar las sentencias que se encuentran entre:

#------------------------------------------------------------------------------------------------------------------------------

         Modifique SOLO aquí dentro
#------------------------------------------------------------------------------------------------------------------------------

Importante:
- También se recomienda correr una a una las celdas, y no correr de nuevo una celda mientras se está ejecutando (el software puede bloquearse). Si mientras una celda se está ejecutando, se realiza algún cambio en el código de esa celda y queremos volver a ejecutarla, deberemos esperar a que termine o reiniciar los kernels.
- Si lo desea, puede jugar con el resto del código.
- Para el desarrollo de la práctica y elaboración del informe se pueden emplear resultados obtenidos en otras prácticas, que sean relativos a la misma situación atmosférica.

**IMPORTACIÓN DE LIBRERÍAS Y ESPECIFICACIÓN DE RUTAS**

En primer lugar **importamos las librerias** necesarias. En el caso de que no se encuentre alguno de los módulos al correr esta celda tendremos que instalar dicho módulo en Anaconda Navigator -> Environments o a traves de Anaconda Prompt.

In [ ]:
import os
from pathlib import Path
from pdb import set_trace

from datetime import datetime, date
import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.integrate import trapz
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.atmo import ecmwf, atmo

from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import date_quicklook, quicklook_xarray
from gfatpy.lidar.quality_assurance.rayleigh_fit import rayleigh_fit_from_date
from gfatpy.lidar import utils
from gfatpy.lidar.retrieval import klett
from gfatpy.lidar.retrieval.raman import raman_extinction
from gfatpy.lidar.retrieval.helper import RetrievalHelper

from gfatpy.aeronet.reader import reader as aeronet_reader
from gfatpy.aeronet.utils import find_aod

from gfatpy.utils.plot import font_axes, color_list


%load_ext autoreload
%autoreload 2

%matplotlib inline

Tambien, antes de comenzar con el tratamiento de los ficheros, definimos la ruta en la que se encuentran los datos.

In [ ]:
#PC GFAT
#------------------------------------------------------------------------------------------------------------------------------
databasepath = Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_raman\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
outputpath =   Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_raman')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

#Portátil 
#------------------------------------------------------------------------------------------------------------------------------
# databasepath = Path(r'G:\My Drive\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_raman\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
# outputpath =   Path(r'G:\My Drive\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_raman')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

#GFATPY
#------------------------------------------------------------------------------------------------------------------------------
# databasepath = Path() / '1a'  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
# outputpath =   Path()  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

databasepath.is_dir(), outputpath.is_dir() #Comprueba que ambas carpetas existen.

En esta práctica, utilizaremos los datos del 10-06-2021:

In [ ]:
# Lectura de datos
#------------------------------------------------------------------------------------------------------------------------------
filepath = databasepath / '2021'/ '06'/ '10' / 'mhc_1a_Prs_rs_xf_20210610.nc'   #Lidar product path. Example : 'C:\\Users\\user\\Desktop\\data\\1a\\2020\\01\\01\\mhc_1a_Prs_rs_xf_20200101.nc' 
dcfilepath = databasepath / '2021'/ '06'/ '10' / 'mhc_1a_Pdc_rs_xf_20210610_1828.nc' #Lidar dark current product path
ini_date = '20210610T220000'   #Initial date and time. Format 'yyyymmddThhmmss', where T represents literally the letter T. Example: '20200101T200000'
end_date = '20210610T230000'   #Same format than ini_date
#------------------------------------------------------------------------------------------------------------------------------

channels = ['355xta', '354xtp']
if os.path.exists(databasepath) and os.path.exists(dcfilepath):    
    data = preprocess(filepath, channels=channels, crop_ranges=(0,20000),gluing_products=False)
else:
    print("One (or both) of the given files can not be found.")

**REPRESENTACIÓN DE SEÑAL CORREGIDA DE RANGO Y RETRODISPERSIÓN MOLECULAR**

Para hacernos una idea de la situación atmosférica, y también para poder identificar la región libre de partículas de aerosol atmosférico, se representa **la evolución temporal de la señal corregida de rango (RCS)**.

Para ello, se necesita:
- `data.signal_355xta`: señal lidar a 355 nm.
- Función `quicklook_xarray` donde `is_rcs` indica que los datos facilitados NO están corregidos de rango, así el programa realiza previamente este cálculo.
- `ymin` e `ymax` son las alturas mínima y máxima  en metros asl, respectivamente, para representar el perfil de RCS. Son parámetros opcionales, pero pueden ser útiles.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
qlpath = outputpath / 'quicklooks' #Output quicklook folder path. It is suggested to mantain the proposed path
ymin = 0
ymax = 7500
#------------------------------------------------------------------------------------------------------------------------------
fig, ax = quicklook_xarray(data.signal_355xta,is_rcs=False)
ax.set_ylim(ymin, ymax)

A continuación realizaremos el Rayleigh fit ---comparación del perfil de la RCS normalizado junto con el de retrodispersión molecular normalizado---. 

Para ello definimos los parámetros de entrada que nos ayudan a determinar el perfil de la RCS normalizado:
- `min_profile_range`, `max_profile_range`: alturas que definen el perfil a utilizar en los cálculos. 
- `min_reference_height`, `max_reference_height`: las alturas en metros asl entre las cuales normalizar el perfil de la RCS y el de retrodispersión molecular, de manera que encierren atmósfera molecular. Lo ideal es utilizar una ventana de un kilómetro, aunque puede ser interesante probar diferentes situaciones. 
- `signal_355xta`: es el perfil lidar elástico a 355 nm en modo analógico.
- `rcs_355xta`: RCS de `signal_355xta`.
- `rcs_355xta_profile`: promedio en el tiempo definido entre `ini_date_str` y `end_date_str`.
- `nrcs_355xta`: perfil normalizado por el valor promedio entre `min_reference_height` y `max_reference_height`.
- `signal_354xtp`: es el perfil lidar Raman a 354 nm en modo de recuento de fotones.
- `rcs_354xtp`: como `rcs_355xta` pero con `signal_354xtp`.
- `rcs_354xtp_profile`: como `rcs_355xta_profile` pero con `signal_354xtp`.
- `nrcs_354xtp`: como `nrcs_355xta` pero con `signal_354xtp`.

Consejo: `max_profile_range` > `y_norm_max`.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
min_reference_height = 6500     #Sets the value of the minimum reference altitude in meters asl
max_reference_height = 7000    #Sets the value of the maximum reference altitude in meters asl
max_profile_range = 10000
min_profile_range= 0
#------------------------------------------------------------------------------------------------------------------------------

datestr, hour_ini, hour_end = '20210610', '220000', '233000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
rcs_355xta = data['signal_355xta']*data.range**2
rcs_354xtp = data['signal_354xtp']*data.range**2
rcs_355xta_profile = rcs_355xta.sel(time=slice(ini_date_str,end_date_str), range=slice(min_profile_range,max_profile_range)).mean('time')
rcs_354xtp_profile = rcs_354xtp.sel(time=slice(ini_date_str,end_date_str), range=slice(min_profile_range,max_profile_range)).mean('time')
nrcs_355xta = rcs_355xta_profile/rcs_355xta_profile.sel(range=slice(min_reference_height,max_reference_height)).mean('range')
nrcs_354xtp = rcs_354xtp_profile/rcs_354xtp_profile.sel(range=slice(min_reference_height,max_reference_height)).mean('range')


Par el Rayleigh fit, es necesario el perfil del coeficiente de retrodispersión molecular normalizado. Para obtener el perfil, necesitamos los perfiles de presión (`pressure`) y temperatura (`temperature`). En este caso, los obtenemos del modelo ECMWF, para las alturas del perfil lidar.

In [ ]:
ranges = rcs_355xta_profile.range
meteo_profiles = ecmwf.get_ecmwf_temperature_pressure(
    datestr, heights=ranges
)

temperature = meteo_profiles['temperature'].values
pressure = meteo_profiles['pressure'].values

En la celda siguiente, se calculan las propiedades moleculares a partir de la temperatura y presión del model ECMWF y se normalizan en el mismo rango de alturas.

In [ ]:
# Molecular Attenuated Backscatter
mol_properties = atmo.molecular_properties(wavelength, pressure, temperature, ranges)
beta_mol_att_355 = mol_properties["attenuated_molecular_beta"]
wavelength = 355
nbeta_mol_355 = beta_mol_att_355/beta_mol_att_355.sel(range=slice(min_reference_height,max_reference_height)).mean('range')

wavelength = 354
mol_properties = atmo.molecular_properties(wavelength, pressure, temperature, ranges)
beta_mol_att_354 = mol_properties["attenuated_molecular_beta"]
nbeta_mol_354 = beta_mol_att_354/beta_mol_att_354.sel(range=slice(min_reference_height,max_reference_height)).mean('range')

Con todo lo necesario, se procede con la representación:

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,5))
nrcs_355xta.plot(x='range', label='355xta', ax=ax[0])
nbeta_mol_355.plot(x='range',label='mol',color='k', ax=ax[0])
ax[0].legend()
ax[0].set_ylabel('Norm. signal,\n[a.u.]')
ax[0].set_yscale('log')
ax[0].set_ylim(0.1,10)
ax[0].set_xlim(0.1,10000)

nrcs_354xtp.plot(x='range', label='354xtp', ax=ax[1])
nbeta_mol_354.plot(x='range',label='mol',color='k', ax=ax[1])
ax[1].legend()
ax[1].set_ylabel('Norm. signal,\n[a.u.]')
ax[1].set_yscale('log')
ax[1].set_ylim(0.1,10)
ax[1].set_xlim(0.1,10000)

plt.savefig(outputpath / f'rayleigh_{datestr}.png', dpi=600)

Una vez verificadas las señales Raman y elástica, procedemos a calcular los coeficientes de extinción y retrodispersión con el método Raman.

**MÉTODO RAMAN**

El desarrollo principal de la práctica se basará en las siguientes celdas.

En primer lugar, determinaremos el coeficiente de extinción de partículas $\alpha_{part}$ con la ecuación:

$ \alpha_{part} = \frac{ \frac{d}{dr}ln\left( \frac{N}{RCS_{raman}} \right) - \alpha_{mol}^{raman} - \alpha_{mol}^{elastic}}{1+ \frac{\lambda_{elastic}}{\lambda_{raman}}^k }$

donde se pone de manifiesto que sólo se necesita la RCS de la longitud de onda Raman como medida experimental.

Cálculo de $\alpha_{mol}^{elastic}$ y $\alpha_{mol}^{raman}$:

In [ ]:
#Cálculo molecular
elastic_wavelength, raman_wavelength  = 355, 354
alpha_mol_elastic = atmo.molecular_extinction(elastic_wavelength, pressure, temperature)
alpha_mol_raman = atmo.molecular_extinction(raman_wavelength, pressure, temperature)

Cálculo de la densidad numérica de moléculas de $N_2$, $N$:

In [ ]:
# Calculate number density of the target molecule
number_density = atmo.number_density_at_pt(pressure, temperature).astype(float)

Cálculo del término $ln\left( \frac{N}{RCS_{raman}} \right) $:

In [ ]:
RCS = rcs_354xtp_profile.values.astype(float)
ranges = rcs_354xtp_profile.range.values
range_resolution = ranges[1] - ranges[0]

# Ratio
raw_log_ratio = np.nan * np.ones(len(RCS))
valid_idx = np.logical_and(number_density > 0, RCS > 0) #Necesario porque no se puede realizar el logaritmo de un número negativo.
raw_log_ratio[valid_idx] = np.ma.log(number_density[valid_idx] / RCS[valid_idx])

#Smooth 
window_size_bin = 25
order = 1
log_ratio = savgol_filter(raw_log_ratio, window_size_bin, order, delta=range_resolution, mode="nearest", cval=np.nan)

#Plot
fig, ax = plt.subplots(1,1,figsize=(5,6))
ax.plot(raw_log_ratio, ranges, label='raw')
ax.plot(log_ratio, ranges, label='smooth')
ax.set_ylim(0,7500)
ax.set_xlim(42,45)
ax.set_xlabel('Ratio, [a.u.]')
ax.set_ylabel('Range, [m]')
ax.legend()

Cálculo de la derivada del término calculado anteriormente: $ \frac{d}{dR} ln\left( \frac{N}{RCS_{raman}} \right) $:

Nota: `deriv = 1` es lo que permite el cálculo de la derivada.

In [ ]:
# Calculate 1st derivative     
window_size_m = 250 #meters
order = 1
window_size_bin = np.floor(window_size_m / range_resolution).astype(int)
derivative = savgol_filter(
    log_ratio, window_size_bin, order, deriv=1, delta=range_resolution, mode="nearest", cval=np.nan
)

#Plot
fig, ax = plt.subplots(1,2,figsize=(5,6))
ax[0].plot(raw_log_ratio, ranges, label='raw')
ax[0].plot(log_ratio, ranges, label='smoothed')
ax[0].set_ylim(0,7500)
ax[0].set_xlim(42,45)
ax[0].set_xlabel('Ratio, [a.u.]')
ax[0].legend(loc='upper right')

ax[1].plot(derivative, ranges, label='Derivative')
ax[1].plot(alpha_mol_elastic+alpha_mol_raman, ranges, label=r'$\alpha_{mol}^{elastic} + \alpha_{mol}^{raman} $')

ax[1].set_ylim(0,7500)
ax[1].set_yticks([])
ax[1].set_xlim(-0.001,0.001)
ax[1].set_xlabel(r'Extinction coef, [$m^{-1}]$')
ax[1].legend(loc='upper right')

Finalmente, se obtiene $\alpha_{part}$. En este paso, es necesario: .
- Asumir un exponente de Angstrom (`angstrom_aerosol`). 
- Eliminar la parte del perfil donde se estima que el ruido de la señal es demasiado alto. Este filtrado tiene tres partes:   y `alpha_top_approx`:
    - Por debajo de `alpha_bottom_approx`, se asume que la extinción de partículas es constante.
        - `alpha_part_elastic[ranges<alpha_bottom_range] = alpha_part_elastic[ranges==alpha_bottom_range]`
    - Por encima de `alpha_top_approx`, se asume que la extinción de partículas es cero:
        - `alpha_part_elastic[ranges>alpha_top_approx] = np.nan`
    - Como la extinción negativa no existe, se fuerza a cero donde esto ocurra:
        - `alpha_part_elastic[alpha_part_elastic<0] = 0.`

In [ ]:
# Calculate particle extinction coefficient
angstrom_aerosol = 1

alpha_aer = (derivative - alpha_mol_elastic - alpha_mol_raman) / (
    1 + (elastic_wavelength / float(raman_wavelength)) ** angstrom_aerosol
)

#Remove region with low SNR
#------------------------------------------------------------------------------------------------------------------------------
alpha_bottom_approx =975. 
alpha_top_approx =4200.
#------------------------------------------------------------------------------------------------------------------------------
alpha_bottom_range = rcs_354xtp_profile.range.sel(range=alpha_bottom_approx,method='nearest').values.item()
alpha_top_range = rcs_354xtp_profile.range.sel(range=alpha_top_approx,method='nearest').values.item()
alpha_part_elastic = alpha_aer.copy()
alpha_part_elastic[ranges<alpha_bottom_range] = alpha_part_elastic[ranges==alpha_bottom_range]
alpha_part_elastic[ranges>alpha_top_approx] = np.nan
alpha_part_elastic[alpha_part_elastic<0] = 0.

#Plot
fig, ax = plt.subplots(1,3,figsize=(10,6))
ax[0].plot(log_ratio, ranges, label='smoothed')
ax[0].set_ylim(0,7500)
ax[0].set_xlim(42,45)
ax[0].set_xlabel('Ratio, [a.u.]')
ax[0].set_ylabel('Range, [m]')
ax[0].legend(loc='upper right')

ax[1].plot(derivative*1e6, ranges, label='Derivative')
ax[1].plot(1e6*(alpha_mol_elastic+alpha_mol_raman), ranges, label=r'$\alpha_{mol}^{elastic} + \alpha_{mol}^{raman} $')

ax[1].set_ylim(0,7500)
ax[1].set_yticks([])
ax[1].set_xlim(0,500)
ax[1].set_xlabel(r'$\alpha, [Mm^{-1}]$')
ax[1].legend(loc='upper right')


ax[2].plot(alpha_aer*1e6, ranges,label='raw')
ax[2].plot(alpha_part_elastic*1e6, ranges, c='g',lw=2,label='valid')
ax[2].set_yticks([])
ax[2].set_ylim(0,7500)
ax[2].set_xlim(-10,200)
ax[2].set_xlabel(r'$\alpha_{part}, [Mm^{-1}]$')
ax[2].legend()

Conocido el perfil de extinción de partículas, se puede determinar el coeficiente de retrodispersión de partículas $\beta_{part}$ con la ecuación mostrada en la presentación de la técnica Raman.

Para este cálculo necesitamos:
1. La razón de transmintancias elástica y Raman
1. La razón de señales elástica y Raman
1. El coeficiente de retrodispersión molecular elástico
1. Condición de contorno

**1) La razón de transmintancias elástica y Raman**

Para ello, necesitamos $\alpha_{mol}^{elastic}$, $\alpha_{mol}^{raman}$, $\alpha_{part}^{elastic}$ y $\alpha_{part}^{raman}$. 

Hasta ahora hemos calculado:
- $\alpha_{mol}^{elastic}$ = `alpha_mol_emission`
- $\alpha_{mol}^{raman}$ = `alpha_mol_raman`
- $\alpha_{part}^{elastic}$ = `alpha_part_emission`

Así que nos falta $\alpha_{part}^{raman}$ [`alpha_part_raman`]. Este cálculo se puede realizar haciendo uso del exponente de Angstrom:

In [ ]:
# Calculate profiles of molecular extinction
alpha_part_raman = (alpha_part_elastic* (raman_wavelength / elastic_wavelength) ** -angstrom_aerosol)

Con todas las extinciones calculadas, se pueden calcular las extinciones totales en cada longitud de onda:
- $\alpha_{total}^{elastic} = \alpha_{mol}^{elastic} + \alpha_{part}^{elastic}$ (`alpha_tot_emission`)
- $\alpha_{total}^{raman} = \alpha_{mol}^{raman} + \alpha_{part}^{raman}$ (`alpha_tot_raman`)

In [ ]:
alpha_tot_elastic = alpha_mol_elastic + alpha_part_elastic
alpha_tot_raman = alpha_mol_raman + alpha_part_raman

Lo que nos permite obtener las transimitancias y su razón:
- $T_{total}^{elastic} = \int (\alpha_{mol}^{elastic} + \alpha_{part}^{elastic} ) dr = \int \alpha_{total}^{elastic} dr $


- $T_{total}^{raman} = \int (\alpha_{mol}^{raman} + \alpha_{part}^{raman} ) dr = \int \alpha_{total}^{raman} dr $


- $\frac{T_{total}^{raman}}{ T_{total}^{elastic}} $

In [ ]:
transmittance_raman = atmo.transmittance(alpha_tot_raman, ranges)
transmittance_elastic = atmo.transmittance(alpha_tot_elastic, ranges)

transmittance_ratio = np.exp(-transmittance_raman) / np.exp(-transmittance_elastic)

**2) La razón de señales elástica y Raman**

A continuación, necesitamos la razón de señales elástica y Raman:

In [ ]:
RCS_elastic = rcs_355xta_profile.values.astype(float)
RCS_raman = rcs_354xtp_profile.values.astype(float)
valid_idx = np.logical_and(RCS_raman>0,RCS_elastic>0)
signal_ratio = np.nan*np.ones(len(RCS_elastic))
signal_ratio [valid_idx] =  RCS_elastic[valid_idx]/RCS_raman[valid_idx]

**3) El coeficiente de retrodispersión molecular elástico**

In [ ]:
beta_mol_elastic = mol_properties["molecular_beta"]

**4) Condición de contorno**

Como toda resolución de ecuación diferencial que se precie, requerimos una condición de contorno. En este caso, asumimos que no ha partículas en la región de referencia por lo que `beta_part_at_reference = 0.`.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
min_reference_height = 6500     #Sets the value of the minimum reference altitude in meters asl
max_reference_height = 7000    #Sets the value of the maximum reference altitude in meters asl
beta_part_at_reference = 0.
#------------------------------------------------------------------------------------------------------------------------------
reference_idx = np.logical_and(ranges>min_reference_height, ranges<max_reference_height)
beta_aerosol_at_reference = beta_part_at_reference + np.nanmean(beta_mol_elastic[reference_idx])
bondary_condition_at_reference = beta_aerosol_at_reference*np.nanmean(RCS_raman[reference_idx]/ (RCS_elastic[reference_idx]*number_density[reference_idx]))

Con todos los pasos anteriores, podemos calcular el coeficiente de retrodispersión total en la longitud de onda elástica $\beta_{aer}^{elastic}$:

In [ ]:
beta_aer_elastic = bondary_condition_at_reference * signal_ratio *number_density * transmittance_ratio

Finalmente, para obtener $\beta_{part}^{elastic}$, queda sustraer la contribución molecular ($\beta_{mol}^{elastic}$):

$\beta_{part}^{elastic}$  = $\beta_{aer}^{elastic}$ - $\beta_{mol}^{elastic}$ 

In [ ]:
beta_part_elastic = beta_aer_elastic - beta_mol_elastic

In [ ]:
#Plot
fig, ax = plt.subplots(1,3,figsize=(10,6))
ax[0].plot(transmittance_elastic, ranges, label='Elastic')
ax[0].plot(transmittance_raman, ranges, label='Raman')
ax[0].set_ylim(0,7500)
# ax[0].set_xlim(42,45)
ax[0].set_xlabel('T, [#]')
ax[0].set_ylabel('Range, [m]')
ax[0].legend(loc='upper right')

ax[1].plot(signal_ratio, ranges, label='Derivative')
ax[1].set_ylim(0,7500)
ax[1].set_yticks([])
# ax[1].set_xlim(0,500)
ax[1].set_xlabel(r'$\frac{RCS_{elastic}}{RCS_{raman}}$, [#]')
ax[1].legend(loc='upper right')


ax[2].plot(beta_mol_elastic*1e6, ranges, c='r',lw=1,label='mol')
ax[2].plot(beta_part_elastic*1e6, ranges, c='g',lw=2,label='part')
ax[2].plot(beta_aer_elastic*1e6, ranges, c='b',lw=1,label='aer')
ax[2].set_yticks([])
ax[2].set_ylim(0,7500)
# ax[2].set_xlim(-10,200)
ax[2].set_xlabel(r'$\beta, [Mm^{-1}·sr^{-1}]$')
ax[2].legend()

In [ ]:
#Plot
fig, ax = plt.subplots(1,3,figsize=(10,6))
ax[0].plot(transmittance_elastic, ranges, label='Elastic')
ax[0].plot(transmittance_raman, ranges, label='Raman')
ax[0].set_ylim(0,7500)
# ax[0].set_xlim(42,45)
ax[0].set_xlabel('T, [#]')
ax[0].set_ylabel('Range, [m]')
ax[0].legend(loc='upper right')

ax[1].plot(signal_ratio, ranges, label='Derivative')
ax[1].set_ylim(0,7500)
ax[1].set_yticks([])
# ax[1].set_xlim(0,500)
ax[1].set_xlabel(r'$\frac{RCS_{elastic}}{RCS_{raman}}$, [#]')
ax[1].legend(loc='upper right')


ax[2].plot(beta_mol_elastic*1e6, ranges, c='r',lw=1,label='mol')
ax[2].plot(beta_part_elastic*1e6, ranges, c='g',lw=2,label='part')
ax[2].plot(beta_aer_elastic*1e6, ranges, c='b',lw=1,label='aer')
ax[2].set_yticks([])
ax[2].set_ylim(0,7500)
# ax[2].set_xlim(-10,200)
ax[2].set_xlabel(r'$\beta, [Mm^{-1}·sr^{-1}]$')
ax[2].legend()

Ya que tenemos $\alpha_{part}$ y $\beta_{part}$, podemos calcular el perfil de razón lidar $LR$:

In [ ]:
LR = alpha_part_elastic / beta_part_elastic 



Sin embargo, como el perfil de extinción fue suavizado, lo correcto para calcular $LR$ sería usar un perfil de retrodispersión también suavizado.

In [ ]:
beta_smooth = savgol_filter(beta_part_elastic, window_size_bin, order, delta=range_resolution, mode="nearest", cval=np.nan)
LR_smooth = alpha_part_elastic / beta_smooth

In [ ]:
#Plot
fig, ax = plt.subplots(1,3,figsize=(10,6))
ax[0].plot(alpha_part_elastic*1e6, ranges,label='smooth')
ax[0].set_ylim(0,5000)
ax[0].set_xlim(0,250)
ax[0].set_xlabel(r'$\alpha_{part}, [Mm^{-1}]$')
ax[0].set_ylabel('Range, [m]')
ax[0].legend(loc='upper right')

ax[1].plot(beta_part_elastic*1e6, ranges, c='lightcoral', label='raw')
ax[1].plot(beta_smooth*1e6, ranges, c='darkred',label='smooth')
ax[1].set_ylim(0,5000)
ax[1].set_yticks([])
# ax[1].set_xlim(0,500)
ax[1].set_xlabel(r'$\beta_{part}, [Mm^{-1}]$')
ax[1].legend(loc='upper right')

ax[2].plot(LR, ranges, c='lightcoral',lw=1, label='raw')
ax[2].plot(LR_smooth, ranges, c='darkred',lw=1, label='smooth')
ax[2].set_yticks([]) 
ax[2].set_ylim(0,5000)
ax[2].set_xlim(-10,100)
ax[2].set_xlabel(r'$LR, [sr]$')
ax[2].legend()

Dada la inestabilida de la solución de la extinción, podemos estimar una razón lidar efectiva que poder comparar con la bibliografía:

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
min_average_eight = 2250     #Sets the value of the minimum average altitude in meters asl
max_average_height = 4000    #Sets the value of the maximum average altitude in meters asl
#------------------------------------------------------------------------------------------------------------------------------
average_idx = np.logical_and(ranges>min_average_eight, ranges<max_average_height)
LR_effective = np.nanmean(LR[average_idx])
LR_std = np.nanstd(LR[average_idx])

print('LR[dust] =', np.round(LR_effective,0), '+-', np.round(LR_std,0), 'sr')